# News Article Scraping
- News source for this notebook: Today
-  The keywords to be used for scraping should be "covid" + any mental health keywords found on the google doc + "singapore" (if needed). This notebook extract news article with "covid" and "singapore". Mental health keywords will be filtered in excel. 

In [ ]:
#Using BeautifulSoup 

!conda install beautifulsoup4
!conda install requests


In [10]:
#import necessary libraries
from urllib.request import urlopen
import json
from bs4 import BeautifulSoup
import os
import requests
import pandas as pd
import selenium
from selenium import webdriver


from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import datetime 
import lxml

In [11]:
#initialise dataframes
columns= ["title", "url", "date", "text"]
today_articles= pd.DataFrame(columns= columns)

#search query in Today: covid singapore 
json_url=  "https://www.todayonline.com/json-solr/covid%20singapore/search?page="

search_url= json_url + "0"
response= urlopen(search_url)
data_json= json.loads(response.read())

search_no= 0
hasResult= True 


while hasResult: 
    #instantiate web driver
    driver = webdriver.Chrome()

    for node in data_json['nodes']:
        if node['node']['section'] == 'Singapore': #to extract Singapore-relevant news
            title= node['node']['title']
            print(node['node'])
            url= node['node']['node_url']
            date= datetime.datetime.fromtimestamp(int(node['node']['publication_date'])).strftime('%d/%m/%Y')

            # Load the HTML page
            try:
                driver.get(url)
                # wait for the page to load

                WebDriverWait(driver,15).until(lambda d: d.find_elements_by_css_selector('[id*=article_detail_body]'))


                

            except:
                continue
            
            # Parse processed webpage with BeautifulSoup
            soup = BeautifulSoup(driver.page_source, "html.parser")
            result= soup.find("div", {"id": "article_detail_body"})
            text= result.get_text()



            today_articles.loc[len(today_articles)]= [title, url, date, text]

        else:
            continue
    search_no+=1 
    search_url= json_url + str(search_no)
    print(search_url)
    driver.close()

            #an exception might be thrown, so the code should be in a try-except block
    try:
        #use the browser to get the url. This is suspicious command that might blow up.
        response= urlopen(search_url)
        data_json= json.loads(response.read())
        if data_json['search_count']==null:
            hasResult= False
            
    except Exception as e:                                   # this describes what to do if an exception is thrown
        error_type, error_obj, error_info = sys.exc_info()      # get the exception information
        print ('ERROR FOR LINK:',url)                          #print the link that cause the problem
        print (error_type, 'Line:', error_info.tb_lineno)     #print error info and line that threw the exception
        continue                                              #ignore this page. Abandon this and go back.


{'title': '5-person limit for diners: Business still sluggish at hawker centres in CBD despite Covid-19 rule change', 'node_id': '11907166', 'type': 'Article', 'publication_date': '1626105357', 'updated_date': '1626105912', 'node_url': 'https://admin.todayonline.com/singapore/5-person-limit-diners-business-still-sluggish-hawker-centres-cbd-despite-covid-19-rule', 'author': 'Benjamin Sim\n', 'abstract': 'SINGAPORE — Some hawkers at three hawker centres in the Central Business District (CBD) said that business is just as bad as before or even worse, now that Covid-19 rules were eased for up to five people to dine together at eateries from Monday (July 12). ', 'photo': 'https://www.todayonline.com/sites/default/files/styles/thumbnail_300x225/public/photos/43_images/20210712_133620.jpg?itok=R5B9tS65', 'section': 'Singapore', 'Search result count': '6020'}
{'title': 'Covid-19: MOH probing infections among Vietnamese social hostesses; swab tests for workers at 3 entertainment outlets', 'node

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=91.0.4472.124)


In [16]:
print(today_articles.tail())

                                                  title  \
4337  ‘Misunderstanding’ of Dorscon Orange risk leve...   
4338  Explainer: Why some Covid-19 patients visit do...   
4339  Heroes Unmasked: Love in time of Covid-19 – vo...   
4340  3 new cases of Covid-19 in Singapore, with 2 m...   
4341  Police identify two youths in supermarket drin...   

                                                    url        date  \
4337  http://todayonline.prod.acquia-sites.com/singa...  12/02/2020   
4338  http://todayonline.prod.acquia-sites.com/singa...  12/02/2020   
4339  http://todayonline.prod.acquia-sites.com/singa...  12/02/2020   
4340  http://todayonline.prod.acquia-sites.com/singa...  12/02/2020   
4341  http://todayonline.prod.acquia-sites.com/singa...  12/02/2020   

                                                   text  
4337  SINGAPORE — Despite the Government’s efforts t...  
4338  SINGAPORE — Of the 50 patients who have tested...  
4339  SINGAPORE — The desire to do something

In [17]:
#download as csv file
today_articles.to_csv("pearlyn-TodayNewsArticles.csv")

In [ ]:
today_articles_mental= pd.DataFrame(columns= columns)